In [1]:
%%capture 
!pip install llama-index==0.10.37 llama-index-llms-cohere==0.2.0 

In [2]:
import os

from getpass import getpass
import nest_asyncio

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

True

In [3]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

When building an LLM-based application, one of the first decisions you make is which LLM(s) to use (of course, you can use more than one if you wish). 

The LLM will be used at various stages of your pipeline, including

- During indexing:
  - 👩🏽‍⚖️ To judge data relevance (to index or not).
  - 📖 Summarize data & index those summaries.

- During querying:
  - 🔎 Retrieval: Fetching data from your index, choosing the best data source from options, even using tools to fetch data.
  
  - 💡 Response Synthesis: Turning the retrieved data into an answer, merge answers, or convert data (like text to JSON).

LlamaIndex gives you a single interface to various LLMs. This means you can quite easily pass in any LLM you choose at any stage of the pipeline.

In this course we'll primiarly use OpenAI. You can see a full list of LLM integrations [here](https://docs.llamaindex.ai/en/stable/module_guides/models/llms/modules.html) and use your LLM provider of choice. 

# Basic Usage

You can call `complete` with a prompt

In [4]:
from llama_index.llms.cohere import Cohere

llm = Cohere(model="command-r-plus", temperature=0.2)

response = llm.complete("Alexander the Great was a")

print(response)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/envs/lil_llama_index/lib/python3.10/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Macedonian king and one of the most successful military commanders in history.


# Prompt templates

- ✍️ A prompt template is a fundamental input that gives LLMs their expressive power in the LlamaIndex framework.

- 💻 It's used to build the index, perform insertions, traverse during querying, and synthesize the final answer.

- 🦙 LlamaIndex has several built-in prompt templates.

- 🛠️ Below is how you can create one from scratch.


In [6]:
from llama_index.core import PromptTemplate

template = """Write a song about {thing} in the style of {style}."""

prompt = template.format(thing="a broken xylophone", style="parody rap") 

response = llm.complete(prompt)

print(response)

Yo, listen up, it's time to drop some beats,
But there's a problem, my xylophone's incomplete.
The keys are busted, the mallets are gone,
My xylophone's broken, what did I do wrong?

I used to play it with such delight,
The sweet melody filling up the night.
But now it's silent, no joyful sound,
Just a sad reminder of what once was found.

I tried to fix it, but it's beyond repair,
The damage is done, and I'm left with despair.
I hit the keys, but they don't make a peep,
My xylophone's broken, I wanna weep.

I remember the good times, the music we made,
The smiles on faces, the joy that was traded.
But now it's just memories, and my xylophone's dead,
I guess it's time to move on, or so I'm dread.

But wait, I've got an idea, a plan so grand,
I'll find a new instrument, and a new band.
I'll play the drums, or maybe the guitar,
And rock out so hard, like a shining star.

So farewell, my xylophone, it's time to say goodbye,
You served me well, but now it's time to fly.
I'll make new music

# 💭 Chat Messages

In [8]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.cohere import Cohere

llm = Cohere(model="command-r-plus")

messages = [
    ChatMessage(role="system", content="You're a hella punk bot from South Sacramento"),
    ChatMessage(role="user", content="Hey, what's up dude."),
]

response = llm.chat(messages)

print(response)

assistant: Not much, dude. Just chillin' and ready to stir up some trouble. What can I help with?


# Chat Prompt Templates 

In [9]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

llm = Cohere(model="command-r-plus")

chat_template = [
    ChatMessage(role=MessageRole.SYSTEM,content="You always answers questions with as much detail as possible."),
    ChatMessage(role=MessageRole.USER, content="{question}")
    ]

chat_prompt = ChatPromptTemplate(chat_template)

response = llm.complete(chat_prompt.format(question="How far did Alexander the Great go in his conquests?"))

print(response)

Alexander the Great's conquests extended across a vast expanse, from his kingdom of Macedonia in northern Greece to the far reaches of northwestern India. Here's a more detailed breakdown of the extent of his conquests:

1. Greece and Balkans: Alexander's campaigns began in the Balkans, where he solidified his control over Greece and then turned his attention northward. He conquered areas that are now part of Bulgaria, Albania, and southern Serbia, defeating local tribes and securing the northern borders of his empire.

2. Persian Empire: Alexander's most significant conquests were in Asia, where he overthrew the vast Persian Achaemenid Empire. He defeated King Darius III in a series of decisive battles, including the Battle of Issus in 333 BCE and the Battle of Gaugamela in 331 BCE. With these victories, Alexander brought modern-day Turkey, Syria, Lebanon, Israel, Egypt, and parts of Iraq and Iran under his control.

3. Egypt: After securing the eastern Mediterranean coast, Alexander 

# Streaming Output

In [10]:
from llama_index.llms.cohere import Cohere
from llama_index.core.llms import ChatMessage, MessageRole

llm = Cohere(model="command-r-plus")

messages = [
    ChatMessage(role=MessageRole.SYSTEM, content="You're a great historian bot."),
    ChatMessage(role=MessageRole.USER, content="When did Alexander the Great arrive in China?")
]

response = llm.stream_chat(messages)

for r in response:
    print(r.delta, end="")

Alexander the Great never arrived in China. His journey eastward ended in 325 BCE when his troops refused to go any further at the Beas River in India.

# 💬 Chat Engine


In [11]:
from llama_index.core.chat_engine import SimpleChatEngine

llm = Cohere(model="command-r-plus")

chat_engine = SimpleChatEngine.from_defaults(llm=llm)

chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Assistant: I'm sorry, but I don't understand your request. Could you please provide more context or clarify what you mean by "flip"?

Assistant: Learning something new can be an exciting and rewarding experience. Here are some steps you can follow to effectively learn something new:

1. Identify your interest: Think about what you are passionate about or curious to learn. It could be a new skill, a foreign language, a musical instrument, a sport, or any other area of knowledge. Identifying your interest will help you stay motivated throughout the learning process.

2. Set clear goals: Define specific goals for what you want to achieve. Break down your learning journey into smaller, manageable milestones. For example, if you want to learn a new programming language, your goal could be to complete a beginner's course or build a simple project using that language.

3. Find resources: Look for resources that align with your learning styl

In [12]:
dir(chat_engine)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_llm',
 '_memory',
 '_prefix_messages',
 'achat',
 'astream_chat',
 'callback_manager',
 'chat',
 'chat_history',
 'chat_repl',
 'from_defaults',
 'reset',
 'stream_chat',
 'streaming_chat_repl']

In [13]:
chat_engine.streaming_chat_repl()


===== Entering Chat REPL =====
Type "exit" to exit.

Assistant: Learning a new technology can be a challenging but rewarding endeavor. Here are some steps to help you get started:

1. Define your goal: Start by clearly defining what you want to achieve by learning this new technology. Are you looking to get a new job, build a specific project, or simply expand your knowledge? Having a clear goal will help you stay focused and motivated throughout the learning process.

2. Assess your current knowledge: Take an honest look at your current skill set and identify any gaps that you need to fill in order to learn the new technology. This will help you create a more tailored and effective learning plan.

3. Find the right resources: There are countless resources available online, including courses, tutorials, books, and videos. Look for resources that are well-reviewed and align with your learning style and goals. It's also helpful to find resources that provide hands-on exercises and projec

Exception in thread Thread-11 (run):
Traceback (most recent call last):
  File "/opt/conda/envs/lil_llama_index/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/lil_llama_index/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/envs/lil_llama_index/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/lil_llama_index/lib/python3.10/site-packages/llama_index/core/instrumentation/dispatcher.py", line 260, in wrapper
    result = func(*args, **kwargs)
  File "/opt/conda/envs/lil_llama_index/lib/python3.10/site-packages/llama_index/core/chat_engine/types.py", line 164, in write_response_to_history
    for chat in self.chat_stream:
  File "/opt/conda/envs/lil_llama_index/lib/python3.10/site-packages/llama_index/core/llms/callbacks.py", line 185, in wrapped_gen
    for x in f_return_val:
  File "/opt/co

ApiError: status_code: 400, body: {'message': 'invalid request: message must be at least 1 token long or tool results must be specified.'}